In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({"key1": ["a", "a", None, "b", "b", "a", None], 
                  "key2": pd.Series([1, 2, 1, 2, 1, None, 1], dtype = "Int64"), 
                  "data1": np.random.standard_normal(7), 
                  "data2": np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,1.059624,2.031095
1,a,2,-0.728861,1.054062
2,None,1,1.096692,-1.404325
3,b,2,0.827754,-0.839391
4,b,1,0.167763,1.128426
5,a,<NA>,-0.489648,-0.101139
6,None,1,0.022564,-1.097066


In [4]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [5]:
grouped.mean()

key1
a   -0.014481
b    0.407311
Name: data1, dtype: float64

In [8]:
df["data1"].groupby([df["key1"], df["key2"]]).mean()

key1  key2
a     1       0.306946
      2      -0.801689
b     1       0.869221
      2      -0.054599
Name: data1, dtype: float64

In [10]:
df["data1"].groupby([df["key1"], df["key2"]]).mean().unstack(level = 1)

key2,1,2
key1,,
a,0.306946,-0.801689
b,0.869221,-0.054599


In [11]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

df["data1"].groupby([states, years]).mean()

CA  2005   -0.175194
    2006    0.325241
OH  2005    0.126173
    2006    1.166867
Name: data1, dtype: float64

In [12]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,-0.014481,0.518838
b,1.5,0.407311,-0.341854


In [17]:
df.groupby(["key1", "key2"]).mean().sort_index(level=0)

data1     data2
key1 key2                    
a    1     0.306946  1.077359
     2    -0.801689  0.802102
b    1     0.869221  0.194550
     2    -0.054599 -0.878258

In [18]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [19]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [20]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


## Iterating over Groups
The object returned by groupby supports iteration

In [23]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1  0.306946  1.077359
1    a     2 -0.801689  0.802102
5    a  <NA>  0.451301 -0.322948
b
  key1  key2     data1     data2
3    b     2 -0.054599 -0.878258
4    b     1  0.869221  0.194550


In [24]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', 1)
  key1  key2     data1     data2
0    a     1  0.306946  1.077359
('a', 2)
  key1  key2     data1     data2
1    a     2 -0.801689  0.802102
('b', 1)
  key1  key2     data1    data2
4    b     1  0.869221  0.19455
('b', 2)
  key1  key2     data1     data2
3    b     2 -0.054599 -0.878258


In [26]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["a"]

,key1,key2,data1,data2
0,a,1,0.306946,1.077359
1,a,2,-0.801689,0.802102
5,a,<NA>,0.451301,-0.322948


In [27]:
pieces["b"]

,key1,key2,data1,data2
3,b,2,-0.054599,-0.878258
4,b,1,0.869221,0.194550


In [28]:
pieces

{'a':   key1  key2     data1     data2
 0    a     1  0.306946  1.077359
 1    a     2 -0.801689  0.802102
 5    a  <NA>  0.451301 -0.322948,
 'b':   key1  key2     data1     data2
 3    b     2 -0.054599 -0.878258
 4    b     1  0.869221  0.194550}

## Selecting a Column or Subset of Columns

In [31]:
df.groupby("key1")["data1"]

In [33]:
df.groupby("key1")[["data2"]]

In [34]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     1.077359
     2     0.802102
b    1     0.194550
     2    -0.878258

## Grouping with Dictionaries and Series

In [35]:
people = pd.DataFrame(np.random.standard_normal((5, 5)), 
                     columns = list("abcde"), 
                     index = ["Joe", "Steve", "Wanda", "Jill", "Trey"])
people

,a,b,c,d,e
Joe,1.088530,-1.094391,1.003585,-0.724700,0.052118
Steve,0.385393,-0.567298,0.507364,0.636190,-0.495307
Wanda,0.319286,1.788023,-1.628611,0.660529,1.369250
Jill,-0.070483,0.576009,-0.660051,-0.949492,-1.198684
Trey,0.023420,-1.224890,1.070576,-0.303935,-1.530953


In [36]:
people.iloc[2:3, [1, 2]] = np.nan

In [37]:
people

,a,b,c,d,e
Joe,1.088530,-1.094391,1.003585,-0.724700,0.052118
Steve,0.385393,-0.567298,0.507364,0.636190,-0.495307
Wanda,0.319286,NaN,NaN,0.660529,1.369250
Jill,-0.070483,0.576009,-0.660051,-0.949492,-1.198684
Trey,0.023420,-1.224890,1.070576,-0.303935,-1.530953


In [38]:
mapping = {"a": "red", 
           "b": "red", 
          "c": "blue", 
          "d": "blue", 
          "e": "red", 
          "f": "orange"}

In [44]:
# groupby index
by_column = people.T.groupby(mapping)
by_column.sum().T

,blue,red
Joe,0.278885,0.046258
Steve,1.143555,-0.677213
Wanda,0.660529,1.688537
Jill,-1.609543,-0.693157
Trey,0.766641,-2.732423


In [45]:
my_series = pd.Series(mapping)
my_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [51]:
people.T.groupby(my_series).count().T

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


## Grouping with Functions

In [55]:
people

,a,b,c,d,e
Joe,1.088530,-1.094391,1.003585,-0.724700,0.052118
Steve,0.385393,-0.567298,0.507364,0.636190,-0.495307
Wanda,0.319286,NaN,NaN,0.660529,1.369250
Jill,-0.070483,0.576009,-0.660051,-0.949492,-1.198684
Trey,0.023420,-1.224890,1.070576,-0.303935,-1.530953


In [57]:
# groupby index
people.groupby(len).sum()

,a,b,c,d,e
3,1.088530,-1.094391,1.003585,-0.724700,0.052118
4,-0.047062,-0.648882,0.410525,-1.253426,-2.729636
5,0.704679,-0.567298,0.507364,1.296719,0.873943


In [58]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,1.088530,-1.094391,1.003585,-0.724700,0.052118
4,two,-0.070483,-1.224890,-0.660051,-0.949492,-1.530953
5,one,0.319286,-0.567298,0.507364,0.636190,-0.495307


## Grouping by Index Levels

In [59]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"], 
                                    [1, 3, 5, 1, 3]],
                                   names = ["cty", "tenor"])
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [61]:
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), 
                      columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.700276  0.272131 -0.100042  0.673105  0.535322
1     -0.792237  1.118145 -0.827537  0.834272 -1.251710
2     -0.595244  0.480024  0.005973  0.161820  1.205095
3     -0.425914  1.062401 -1.115572  0.878103  0.220075

In [64]:
hier_df.T.groupby(level = "cty").count()

,0,1,2,3
cty,,,,
JP,2,2,2,2
US,3,3,3,3


# Data Aggregation
Aggregations refer to any data transformation that produces scalar values from arrays.

In [65]:
df

,key1,key2,data1,data2
0,a,1,0.306946,1.077359
1,a,2,-0.801689,0.802102
2,None,1,0.325241,0.958808
3,b,2,-0.054599,-0.878258
4,b,1,0.869221,0.194550
5,a,<NA>,0.451301,-0.322948
6,None,1,1.464514,-0.667697


In [67]:
grouped = df.groupby("key1")
grouped["data1"].nsmallest(1)

key1   
a     1   -0.801689
b     3   -0.054599
Name: data1, dtype: float64

In [68]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.252989,1.400307
b,1,0.923820,1.072808


In [4]:
grouped.describe()

NameError: name 'grouped' is not defined

## Column-Wise and Multiple Function Application

In [76]:
tips = pd.read_csv("../../github/pydata-book/examples/tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [79]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [81]:
grouped = tips.groupby(["day", "smoker"])
grouped_pct = grouped["tip_pct"]

In [82]:
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [84]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [86]:
grouped_pct.agg([("average", "mean"), ("stdev", "std")])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [88]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [89]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [93]:
ftuples = [("Average", "mean"), ("Variance", "var")]

grouped[["tip_pct", "total_bill"]].agg(ftuples)

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [95]:
# different functions for differet columns
grouped.agg({"tip": "max", "size": "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [96]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"], 
            "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

## Returning Aggregated Data Without Row Indexes

In [108]:
tips = tips[["total_bill", "tip", "smoker", "day", "size", "tip_pct"]]
tips.head()

,total_bill,tip,smoker,day,size,tip_pct
0,16.99,1.01,No,Sun,2,0.059447
1,10.34,1.66,No,Sun,3,0.160542
2,21.01,3.50,No,Sun,3,0.166587
3,23.68,3.31,No,Sun,2,0.139780
4,24.59,3.61,No,Sun,4,0.146808


In [134]:
tips.groupby(["day", "smoker"], as_index=True).mean().reorder_levels(["smoker", "day"]).sort_index(level=0)

total_bill       tip      size   tip_pct
smoker day                                           
No     Fri    18.420000  2.812500  2.250000  0.151650
       Sat    19.661778  3.102889  2.555556  0.158048
       Sun    20.506667  3.167895  2.929825  0.160113
       Thur   17.113111  2.673778  2.488889  0.160298
Yes    Fri    16.813333  2.714000  2.066667  0.174783
       Sat    21.276667  2.875476  2.476190  0.147906
       Sun    24.120000  3.516842  2.578947  0.187250
       Thur   19.190588  3.030000  2.352941  0.163863

## Apply: General split-apply-combine

In [135]:
def top(df, n = 5, column = "tip_pct"):
    return df.sort_values(column, ascending = False)[:n]

top(tips, n = 6)

,total_bill,tip,smoker,day,size,tip_pct
172,7.25,5.15,Yes,Sun,2,0.710345
178,9.60,4.00,Yes,Sun,2,0.416667
67,3.07,1.00,Yes,Sat,1,0.325733
232,11.61,3.39,No,Sat,2,0.291990
183,23.17,6.50,Yes,Sun,4,0.280535
109,14.31,4.00,Yes,Sat,2,0.279525


In [136]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day  size   tip_pct
smoker                                                   
No     232       11.61  3.39     No   Sat     2  0.291990
       149        7.51  2.00     No  Thur     2  0.266312
       51        10.29  2.60     No   Sun     2  0.252672
       185       20.69  5.00     No   Sun     5  0.241663
       88        24.71  5.85     No  Thur     2  0.236746
Yes    172        7.25  5.15    Yes   Sun     2  0.710345
       178        9.60  4.00    Yes   Sun     2  0.416667
       67         3.07  1.00    Yes   Sat     1  0.325733
       183       23.17  6.50    Yes   Sun     4  0.280535
       109       14.31  4.00    Yes   Sat     2  0.279525

In [139]:
tips.groupby(["day", "smoker"]).apply(top, n = 2, column = "total_bill")

total_bill    tip smoker   day  size   tip_pct
day  smoker                                                    
Fri  No     94        22.75   3.25     No   Fri     2  0.142857
            91        22.49   3.50     No   Fri     2  0.155625
     Yes    95        40.17   4.73    Yes   Fri     4  0.117750
            90        28.97   3.00    Yes   Fri     2  0.103555
Sat  No     212       48.33   9.00     No   Sat     4  0.186220
            59        48.27   6.73     No   Sat     4  0.139424
     Yes    170       50.81  10.00    Yes   Sat     3  0.196812
            102       44.30   2.50    Yes   Sat     3  0.056433
Sun  No     156       48.17   5.00     No   Sun     6  0.103799
            112       38.07   4.00     No   Sun     3  0.105070
     Yes    182       45.35   3.50    Yes   Sun     3  0.077178
            184       40.55   3.00    Yes   Sun     2  0.073983
Thur No     142       41.19   5.00     No  Thur     5  0.121389
            85        34.83   5.17     No  Thur     4  0.148435
     Yes    197       43.11   5.00    Yes  Thur     4  0.115982
            83        32.68   5.00    Yes  Thur     2  0.152999

In [147]:
tips.groupby("smoker", group_keys=False).apply(top, n = 3)

,total_bill,tip,smoker,day,size,tip_pct
232,11.61,3.39,No,Sat,2,0.291990
149,7.51,2.00,No,Thur,2,0.266312
51,10.29,2.60,No,Sun,2,0.252672
172,7.25,5.15,Yes,Sun,2,0.710345
178,9.60,4.00,Yes,Sun,2,0.416667
67,3.07,1.00,Yes,Sat,1,0.325733


In [152]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000), 
                     "data2": np.random.standard_normal(1000)})
frame

,data1,data2
0,-1.072977,-0.069353
1,-1.455718,0.689834
2,-0.443467,-1.212230
3,-0.889017,0.299874
4,1.523226,-0.111995
...,...,...
995,-0.261482,-0.897520
996,-1.378638,-1.027089
997,1.098183,1.869100
998,-3.073446,2.070035


In [153]:
quantiles = pd.cut(frame["data1"], 4)
quantiles

0      (-1.668, -0.123]
1      (-1.668, -0.123]
2      (-1.668, -0.123]
3      (-1.668, -0.123]
4        (1.422, 2.967]
             ...       
995    (-1.668, -0.123]
996    (-1.668, -0.123]
997     (-0.123, 1.422]
998     (-3.22, -1.668]
999      (1.422, 2.967]
Name: data1, Length: 1000, dtype: category
Categories (4, interval[float64, right]): [(-3.22, -1.668] < (-1.668, -0.123] < (-0.123, 1.422] < (1.422, 2.967]]

In [158]:
def get_stats(group):
    return pd.DataFrame(
            {"min": group.min(), 
            "max": group.max(), 
            "count": group.count(), 
            "mean": group.mean()})

grouped = frame.groupby(quantiles, observed = True)

In [161]:
grouped.apply(get_stats)

min       max  count      mean
data1                                                      
(-3.22, -1.668]  data1 -3.213622 -1.684120     63 -2.036067
                 data2 -2.919038  2.070035     63  0.138146
(-1.668, -0.123] data1 -1.655951 -0.123838    391 -0.734325
                 data2 -2.937128  2.782455    391 -0.066339
(-0.123, 1.422]  data1 -0.116602  1.414666    471  0.540380
                 data2 -3.218364  2.861852    471  0.012211
(1.422, 2.967]   data1  1.429231  2.966981     75  1.865729
                 data2 -2.016647  1.951432     75 -0.240433

In [162]:
grouped.agg(["min", "max", "count", "mean"])

data1                               data2            \
                       min       max count      mean       min       max   
data1                                                                      
(-3.22, -1.668]  -3.213622 -1.684120    63 -2.036067 -2.919038  2.070035   
(-1.668, -0.123] -1.655951 -0.123838   391 -0.734325 -2.937128  2.782455   
(-0.123, 1.422]  -0.116602  1.414666   471  0.540380 -3.218364  2.861852   
(1.422, 2.967]    1.429231  2.966981    75  1.865729 -2.016647  1.951432   

                                  
                 count      mean  
data1                             
(-3.22, -1.668]     63  0.138146  
(-1.668, -0.123]   391 -0.066339  
(-0.123, 1.422]    471  0.012211  
(1.422, 2.967]      75 -0.240433

In [163]:
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s

0         NaN
1    0.008706
2         NaN
3   -0.163444
4         NaN
5   -0.889703
dtype: float64

In [165]:
s.fillna(value = s.mean())

0   -0.348147
1    0.008706
2   -0.348147
3   -0.163444
4   -0.348147
5   -0.889703
dtype: float64

In [167]:
states = ["Ohio", "New York", "Vermont", "Florida",
"Oregon", "Nevada", "California", "Idaho"]
states

['Ohio',
 'New York',
 'Vermont',
 'Florida',
 'Oregon',
 'Nevada',
 'California',
 'Idaho']

In [168]:
group_key = ["East", "East", "East", "East", 
"West", "West", "West", "West"]
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [170]:
data = pd.Series(np.random.standard_normal(8), index=states)
data

Ohio         -0.517623
New York     -0.468200
Vermont      -0.084308
Florida       0.690280
Oregon        0.363221
Nevada       -0.670056
California    0.032454
Idaho        -0.964436
dtype: float64

In [171]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio         -0.517623
New York     -0.468200
Vermont            NaN
Florida       0.690280
Oregon        0.363221
Nevada             NaN
California    0.032454
Idaho              NaN
dtype: float64

In [175]:
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [176]:
data.groupby(group_key).mean()

East   -0.098514
West    0.197838
dtype: float64

In [177]:
data.groupby(group_key).count()

East    3
West    2
dtype: int64

In [178]:
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

East  Ohio         -0.517623
      New York     -0.468200
      Vermont      -0.098514
      Florida       0.690280
West  Oregon        0.363221
      Nevada        0.197838
      California    0.032454
      Idaho         0.197838
dtype: float64

In [183]:
fill_values = {"East": 0.5, "West": -1}
def fill_mean(group):
    return group.fillna(fill_values[group.name])

data.groupby(group_key).apply(fill_mean)

East  Ohio         -0.517623
      New York     -0.468200
      Vermont       0.500000
      Florida       0.690280
West  Oregon        0.363221
      Nevada       -1.000000
      California    0.032454
      Idaho        -1.000000
dtype: float64

## Random Sampling and Permutation

In [208]:
suits = ["H", "S", "C", "D"]
card_val = (list(range(1, 11)) + [10] * 3) * 4

base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]

cards = []

for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index = cards)
deck.head(13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [209]:
def draw(deck, n = 5):
    return deck.sample(n)

draw(deck)

4D     4
5H     5
JS    10
KH    10
JC    10
dtype: int64

In [225]:
def get_suit(card):
    return card[-1]

deck.groupby(get_suit).apply(draw, n = 2)

C  AC      1
   5C      5
D  10D    10
   AD      1
H  8H      8
   JH     10
S  JS     10
   5S      5
dtype: int64

In [230]:
df = pd.DataFrame({"key": ["a", "b", "c"] * 4, 
                  "value": np.arange(12)})
df

,key,value
0,a,0
1,b,1
2,c,2
3,a,3
4,b,4
5,c,5
6,a,6
7,b,7
8,c,8
9,a,9


In [231]:
df.groupby("key").mean()

,value
key,
a,4.5
b,5.5
c,6.5


In [235]:
def get_mean(group):
    return group.mean()

df.groupby("key").transform(get_mean)

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


## Pivot Tables and Cross-Tabulation

In [236]:
tips.head()

,total_bill,tip,smoker,day,size,tip_pct
0,16.99,1.01,No,Sun,2,0.059447
1,10.34,1.66,No,Sun,3,0.160542
2,21.01,3.50,No,Sun,3,0.166587
3,23.68,3.31,No,Sun,2,0.139780
4,24.59,3.61,No,Sun,4,0.146808


In [237]:
tips.pivot_table(index = ["day", "smoker"])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [240]:
tips.pivot_table(index = ["day"], 
                columns="smoker", 
                values=["tip_pct", "size"], 
                margins=True)

size                       tip_pct                    
smoker        No       Yes       All        No       Yes       All
day                                                               
Fri     2.250000  2.066667  2.105263  0.151650  0.174783  0.169913
Sat     2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
Sun     2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
Thur    2.488889  2.352941  2.451613  0.160298  0.163863  0.161276
All     2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [242]:
from io import StringIO

In [245]:
data = """Sample Nationality Handedness 
1 USA Right-handed
2 Japan Left-handed 
3 USA Right-handed 
4 Japan Right-handed 
5 Japan Left-handed
6 Japan left-handed 
7 Japan Right-handed
8 USA Right-handed 
9 USA Left-handed 
10 Japan Right-handed
11 USA Right-handed"""
data

'Sample Nationality Handedness \n1 USA Right-handed\n2 Japan Left-handed \n3 USA Right-handed \n4 Japan Right-handed \n5 Japan Left-handed\n6 Japan left-handed \n7 Japan Right-handed\n8 USA Right-handed \n9 USA Left-handed \n10 Japan Right-handed\n11 USA Right-handed'

In [246]:
data = pd.read_table(StringIO(data), sep = "\s+")
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,left-handed
6,7,Japan,Right-handed
7,8,USA,Right-handed
8,9,USA,Left-handed
9,10,Japan,Right-handed


In [248]:
pd.crosstab(data["Nationality"], data["Handedness"], margins = True)

Handedness,Left-handed,Right-handed,left-handed,All
Nationality,,,,
Japan,2,3,1,6
USA,1,4,0,5
All,3,7,1,11


In [250]:
pd.crosstab(tips["day"], tips["smoker"], margins = True)

smoker,No,Yes,All
day,,,
Fri,4,15,19
Sat,45,42,87
Sun,57,19,76
Thur,45,17,62
All,151,93,244
